In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text.RegularExpressions;

var data = File.ReadAllText("inputs/input_day18.txt");

In [ ]:
// Test data
var dataT = 
//@"[[[[[9,8],1],2],3],4]";

"[7,[6,[5,[4,[3,2]]]]]";

In [ ]:
Pair Parse(string snailfishNumber)
{
    var stack = new Stack<Pair>();

    var number = "";

    foreach(var ch in snailfishNumber)
    {
        if(ch == '[')
        {
            stack.Push(new Pair());
            continue;
        }

        if(char.IsDigit(ch))
        {
            number += ch;
            continue;
        }

        if(ch == ',')
        {
            if(number.Length > 0)
            {
                stack.Push(new Pair(int.Parse(number)));
                number = "";
            }
        }

        if(ch == ']')
        {
            if(number.Length > 0)
            {
                stack.Push(new Pair(int.Parse(number)));
                number = "";
            }
            
            var right = stack.Pop();
            var left = stack.Pop();
            var pair = stack.Peek();
            pair.SetLeftRight(left, right);
        }
    }

    return stack.Pop();
}


In [ ]:
internal class Pair
{
    private bool _isRegular;

    private int _regular;

    public int Value => _regular;

    public bool AreChildrenRegular => !_isRegular && Left._isRegular && Right._isRegular;

    private Pair Parent { get; set; }

    public Pair Left { get; set; }
    public Pair Right { get; set; }

    public Pair(Pair left, Pair right)
    {
        Left = left;
        Right = right;

        Left.Parent = this;
        Right.Parent = this;
    }

    public Pair()
    { }

    public Pair(int value)
    {
        _regular = value;
        _isRegular = true;
    }

    public void SetLeftRight(Pair left, Pair right)
    {
        Left = left;
        Right = right;

        Left.Parent = this;
        Right.Parent = this;
    }

    public void IncrementValue(int amount)
    {
        if (!_isRegular)
        {
            return;
        }

        _regular += amount;
    }

    private void ReplaceByRegular0()
    {
        Left = null;
        Right = null;
        _regular = 0;
        _isRegular = true;
    }

    private void Split()
    {
        Left = new Pair(Value / 2);
        Right = new Pair((int)Math.Ceiling(Value / 2.0));
        Left.Parent = this;
        Right.Parent = this;

        _regular = 0;
        _isRegular = false;
    }

    private (int, Pair) RecurseForDepth(Pair pair, int depth) =>
        pair.AreChildrenRegular
            ? (depth, pair)
            : new[] { 
                pair.Left != null ? RecurseForDepth(pair.Left, depth + 1) : (0, null),
                pair.Right != null ? RecurseForDepth(pair.Right, depth + 1) : (0,null)
            }.MaxBy(_ => _.Item1);

    public Pair FindLeftNeighbour()
    {
        var current = this;

        while (current != null)
        {
            if (current == current.Parent?.Right)
            {
                return current.Parent.Left.FindRegularChild(right: true);
            }

            current = current.Parent;
        }

        return null;
    }

    public Pair FindRightNeighbour()
    {
        var current = this;
        while (current != null)
        {
            if (current == current.Parent?.Left)
            {
                return current.Parent.Right.FindRegularChild(right: false);
            }

            current = current.Parent;
        }

        return null;
    }

    public Pair FindRegularChild(bool right = false)
    {
        var current = this;

        while (true)
        {
            if (current._isRegular)
            {
                return current;
            }

            current =
                right
                    ? current.Right
                    : current.Left;
        }
    }

    public bool DoExplode()
    {
        var (depth, pair) = RecurseForDepth(this, 1);

        if (depth == 5)
        {
            var leftNeighbour = pair.FindLeftNeighbour();
            var rightNeighbour = pair.FindRightNeighbour();

            if (leftNeighbour != null)
            {
                leftNeighbour.IncrementValue(pair.Left.Value);
            }

            if (rightNeighbour != null)
            {
                rightNeighbour.IncrementValue(pair.Right.Value);
            }

            pair.ReplaceByRegular0();

            return true;
        }

        return false;
    }

    private void RecurseForSplit(IList<Pair> list, Pair pair)
    {
        if (pair._isRegular && pair.Value >= 10)
        {
            list.Add(pair);
        }

        if (!pair._isRegular)
        {
            RecurseForSplit(list, pair.Left);
            RecurseForSplit(list, pair.Right);
        }
    }

    private Pair FindFirstNumberToSplitRecurse(Pair pair) =>
        (pair._isRegular && pair.Value >= 10)
            ? pair
            : pair._isRegular
                ? null
                : FindFirstNumberToSplitRecurse(pair.Left) ??
                    FindFirstNumberToSplitRecurse(pair.Right);

    private int MagnitudeRecurse(Pair pair) =>
        3 * (pair.Left._isRegular ? pair.Left.Value : MagnitudeRecurse(pair.Left)) + 
        2 * (pair.Right._isRegular ? pair.Right.Value : MagnitudeRecurse(pair.Right));

    public int Magnitude => MagnitudeRecurse(this);

    public bool DoSplit()
    {
        var pair = FindFirstNumberToSplitRecurse(this);

        if(pair == null)
        { 
            return false; 
        }

        pair.Split();

        return true;
    }

    public void ApplyActions()
    {
        while (true)
        {
            if (!DoExplode() && !DoSplit())
            {
                break;
            }
        }
    }

    public override string ToString() =>
        _isRegular
            ? _regular.ToString()
            : $"[{Left},{Right}]";
}

In [ ]:
var list = data
    .Split("\n")
    .Where(_ => !string.IsNullOrWhiteSpace(_));

Console.WriteLine(list.Count());

var pair = Parse(list.First());
pair.ApplyActions();
Console.WriteLine(pair);

list.Skip(1).ToList()
    .ForEach(line =>
    {
        var newPair = Parse(line);
        newPair.ApplyActions();

        var sum = new Pair();
        sum.SetLeftRight(pair, newPair);

        sum.ApplyActions();

        pair = sum;
    });

Console.WriteLine(pair);

Console.WriteLine(pair.Magnitude);

100
[[1,[8,[5,8]]],[[4,4],[8,[8,8]]]]
[[[[0,6],[7,7]],[[7,8],[6,8]]],[[[8,7],[8,8]],[[1,9],[2,2]]]]
3524
